In [89]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [90]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [91]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [92]:
# create engine to hawaii.sqlite
database_path = "hawaii.sqlite"
engine = create_engine("sqlite:///{}".format(database_path))
print(engine)

Engine(sqlite:///hawaii.sqlite)


In [93]:
# reflect an existing database into a new model
Base = automap_base()

In [94]:

# reflect the tables
Base.prepare(engine, reflect=True)
print(Base.classes.keys())


['measurement', 'station']


In [95]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [96]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [97]:
data = engine.execute("SELECT * FROM Measurement LIMIT 5")
for record in data:
    print(record)

(1, 'USC00519397', '2010-01-01', 0.08, 65.0)
(2, 'USC00519397', '2010-01-02', 0.0, 63.0)
(3, 'USC00519397', '2010-01-03', 0.0, 74.0)
(4, 'USC00519397', '2010-01-04', 0.0, 76.0)
(5, 'USC00519397', '2010-01-06', None, 73.0)


In [98]:
# Find the most recent date in the data set.
most_recent_date= session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(most_recent_date)

('2017-08-23',)


In [123]:
latest_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
latest_date = str(latest_date)
print(latest_date)

2016-08-23


In [130]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column

query = "SELECT * FROM Measurement"  

measurement_df = pd.read_sql_query(query, engine, index_col="id")  
measurement_df.head()

,station,date,prcp,tobs
id,,,,
1,USC00519397,2010-01-01,0.08,65.0
2,USC00519397,2010-01-02,0.00,63.0
3,USC00519397,2010-01-03,0.00,74.0
4,USC00519397,2010-01-04,0.00,76.0
5,USC00519397,2010-01-06,NaN,73.0


In [132]:
#create our filter for dates
filter = (measurement_df['date'] >= '2016-08-23') & (measurement_df['date'] <= '2017-08-23')

In [133]:
# Sort the dataframe by date
measurement_df = measurement_df.loc[filter]
measurement_df

,station,date,prcp,tobs
id,,,,
2364,USC00519397,2016-08-23,0.00,81.0
2365,USC00519397,2016-08-24,0.08,79.0
2366,USC00519397,2016-08-25,0.08,80.0
2367,USC00519397,2016-08-26,0.00,79.0
2368,USC00519397,2016-08-27,0.00,77.0
...,...,...,...,...
19546,USC00516128,2017-08-19,0.09,71.0
19547,USC00516128,2017-08-20,NaN,78.0
19548,USC00516128,2017-08-21,0.56,76.0


In [134]:
measurement_to_plot_df = measurement_df.drop(columns=['station', 'tobs'])
measurement_to_plot_df

,date,prcp
id,,
2364,2016-08-23,0.00
2365,2016-08-24,0.08
2366,2016-08-25,0.08
2367,2016-08-26,0.00
2368,2016-08-27,0.00
...,...,...
19546,2017-08-19,0.09
19547,2017-08-20,NaN
19548,2017-08-21,0.56


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()